In [1]:
import multiprocessing
import time
import numpy as np
import scipy as sp
import emcee
import testkit
import ciamod
import os
import gc
import sys
import pickle
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# set up the model arguments the drop these into theta(state vector) or runargs

# set up pressure grids in bar cos its intuitive
logcoarsePress = np.arange(-4.0, 2.5, 0.53)
#logcoarsePress = np.arange(-4.0, 3.0, 0.5)
coarsePress = pow(10,logcoarsePress)
logfinePress = np.arange(-4.0, 2.4, 0.1)
finePress = pow(10,logfinePress)
# forward model wants pressure in mbar
press = finePress
nprof = coarsePress.size
print nprof
print finePress.size

13
64


In [3]:
w1 = 1.0
w2 = 2.50
npatches = 1
pcover = np.array([npatches],dtype='f')
pcover[:] = 1.0
do_clouds = np.array([npatches],dtype='i')
do_clouds[:] = 0
cloudnum = np.array([npatches],dtype='i')
cloudnum[:] = 99
cloudtype = np.array([npatches],dtype='i')
cloudtype[:] = 1

use_disort = 0 
proftype = 2
dist = 11.84
# now the linelist
# Set up number of gases, and point at the lists. see gaslist.dat
ngas = 10
gasnum = np.asfortranarray(np.array([1,4,7,8,10,11,3,2,20,21],dtype='i'))
lists = ["../Linelists/H2O_xsecs.pic","../Linelists/co_xsecs.pic","../Linelists/tio_xsecs.pic","../Linelists/vo_xsecs.pic","../Linelists/crh_xsecs.pic" ,"../Linelists/feh_xsecs.pic","../Linelists/h2s_xsecs.pic","../Linelists/ch4_xsecs.pic","../Linelists/K_Mike_xsecs.pic","../Linelists/Na_Mike_xsecs.pic"]

In [10]:
# get the basic framework from water list
rawwavenum, inpress, inlinetemps, inlinelist = pickle.load( open('..//Linelists/H2O_xsecs.pic', "rb" ) )
wn1 = 10000./w2
wn2 = 10000. / w1
inwavenum = np.asfortranarray(rawwavenum[np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1)))],dtype='float64')
ntemps = inlinetemps.size
npress= finePress.size
nwave = inwavenum.size
r1 = np.amin(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))
r2 = np.amax(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))

# Here we are interpolating the linelist onto our fine pressure scale.
# pickles have linelist as 4th entry....
linelist = (np.ones([ngas,npress,ntemps,nwave],order='F')).astype('float64', order='F')
for gas in range (0,ngas):
    inlinelist= pickle.load( open(lists[gas], "rb" ) )[3]
    for i in range (0,ntemps):
        for j in range (r1,r2+1):
            pfit = interp1d(np.log10(inpress),np.log10(inlinelist[:,i,j]))
            linelist[gas,:,i,(j-r1)] = np.asfortranarray(pfit(np.log10(finePress)))

linelist[np.isnan(linelist)] = -50.0


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in log10


In [11]:
# cloudparams is structured array with 5 entries
# each one has a patch*cloud entries
cloudparams = np.ones(5)
# 5 entries in cloudparams for simple slab model are:
# 0) log10(number density)
# 1) top layer id (or pressure)
# 2) base ID (these are both in 61 layers)
# 3) rg
# 4) rsig
cloudparams[0] = -20.
cloudparams[1] = 10
cloudparams[2] = 12
cloudparams[3] = 1e-4
cloudparams[4] = 1e-5
# hardwired gas and cloud IDs
cloudnum = np.array([1],dtype='i')



In [12]:
# Get the cia bits
tmpcia, ciatemps = ciamod.read_cia("CIA_DS_aug_2015.dat",inwavenum)
cia = np.asfortranarray(np.empty((4,ciatemps.size,nwave)),dtype='float32')
cia[:,:,:] = tmpcia[:,:,:nwave] 
ciatemps = np.asfortranarray(ciatemps, dtype='float32')


In [13]:
# hardwired FWHM of data
fwhm = 0.005
#fixvmrs = -8.0

# get the observed spectrum
obspec = np.asfortranarray(np.loadtxt("2M2224_mkoJcalib.dat",dtype='d',unpack='true'))

# NEXT LINE ADDS SYTEMATIC TO SPECTRUM FOR Log F retrieval
#obspec[1,:] = obspec[1,:] + 0.1*(min(obspec[2,10::3]**2))


runargs = dist,pcover, cloudtype,cloudparams,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype



In [14]:
nprof = 5
# for proftype = 2 with no clouds, ndim = nprof + ngas-1, + (dlam,beta,logg, r2d2)
ndim =(nprof + (ngas-1) + 4)
theta = np.empty(ndim)
print ndim

18


In [18]:
theta[0] = (0.5*np.random.randn()) - 3.0 # H2O
theta[1] = (0.5*np.random.randn()) - 3.0 # CO
theta[2] = (0.5*np.random.randn()) - 8.0 # TiO
theta[3] = (0.5*np.random.randn()) - 8.0 # VO
theta[4] = (1.0*np.random.randn()) - 6.0 # CrH
theta[5] = (0.5*np.random.randn()) - 4.0 # FeH
theta[6] = (0.5*np.random.randn()) - 5.0 # H2
theta[7] = (0.5*np.random.randn()) - 8.5 # CH4
theta[8] = (0.5*np.random.randn()) - 5.5 # Na+K
#theta[4] = (3*np.random.rand()) # -19. # log ref density
#theta[5] = 100. * np.random.rand() #cloud top
#theta[6] = 100. * np.random.rand() #cloud bottom
#  radii in um
#theta[7] = np.random.rand()# + 10.# rg
#theta[8] = 0.5 * np.random.randn() # rsig
theta[9] = np.random.rand() + 4.2  # logg
theta[10] =   1e-19 * np.random.rand()   # r2d2
theta[11] = np.random.randn() * 0.001 # dlam
theta[12] = np.log10((np.random.rand() * (max(obspec[2,:]**2)*(10. - 0.01))) + (0.01*min(obspec[2,10::3]**2)))
#BTprof = np.loadtxt("BTtemp800_45_13.dat")
theta[13] = np.random.rand()
theta[14] = np.random.rand()
theta[15] =  1e-4 + (np.random.rand() * 10.)
theta[16] =  theta[15] + np.random.rand() * 100.
theta[17] = np.random.rand() * 3000.0

In [19]:
print theta

[ -2.79102526e+00  -3.36016183e+00  -7.75686931e+00  -8.70907527e+00
  -6.36514211e+00  -3.77592522e+00  -4.96629211e+00  -8.70750593e+00
  -5.85470809e+00   4.64533739e+00   7.45921985e-20  -4.42227900e-04
  -2.92681762e+01   6.90848474e-01   8.68455263e-01   9.07627384e+00
   8.69241414e+01   3.95023289e+02]


In [20]:
print testkit.lnprob(theta,*runargs) #* -2. / (obspec.shape[1] /3.)
#print testkit.lnprob(theta2,*runargs) * -2. / (obspec.shape[1] /3.)


[[  2.50000000e+00   2.49937516e+00   2.49875062e+00 ...,   1.00020004e+00
    1.00010001e+00   1.00000000e+00]
 [  4.27959595e-21   3.02390603e-21   4.13873720e-21 ...,   4.93761687e-49
    5.86351031e-57   9.94539133e-55]]
4208.64596575


In [9]:
print pickle.load(open("../Linelists/H2O_xsecs.pic","rb"))[2]

[   75.    85.   100.   120.   140.   160.   180.   200.   230.   260.
   300.   350.   400.   500.   650.   800.  1000.  1200.  1400.  1600.
  1800.  2000.  2300.  2600.  3000.  3500.  4000.]


In [ ]:
print linelist